In [1]:
import pandas as pd
import numpy as np

In [2]:
def add_rows_for_algo(df, row, idx):
    """
    If an algo has multiple variations (delimiter is '; '), split its row into multiple rows
    """
    df = df.drop([idx])
    vars = row['Variation'].split('; ')
    for var in vars:
        copy = row.copy(deep=True)
        copy['Variation'] = var
        df = pd.concat([df, copy.to_frame().T], ignore_index=True)
    return df

In [3]:
def clean_data():
    """
    Go through data.csv and if any algos have multiple variations, rewrite data.csv with the algo split into multiple rows.
    Get rid of trailing ?'s in the complexity class columns.
    """
    dataframe = pd.read_csv('Analysis/data_dirty.csv')
    dataframe = dataframe.replace(np.nan, '', regex=True)
    dataframe = dataframe[(dataframe['Time Complexity Class'] != '#VALUE!') &
                          (dataframe['Space Complexity Class'] != '#VALUE!') &
                          (dataframe["Looked at?"] != 0.001) &
                          (dataframe["Exact Problem Statement?"] == 1)]

    # Get rid of parallel, quantum, and approximate algorithms
    dataframe = dataframe[(dataframe["Quantum?"] == 0) | (dataframe["Quantum?"] == "0")]
    dataframe = dataframe[(dataframe["Parallel?"] == 0) | (dataframe["Parallel?"] == "0")]
    dataframe = dataframe[(dataframe["Approximate?"] == 0) | (dataframe["Approximate?"] == "0")]

    searching = True
    found = False
    while searching:
        for index, row in dataframe.iterrows():
            # if row["Looked at?"] == "0.001" or row["Exact Problem Statement?"] == "0":
            #     continue
            row['Space Complexity Class'] = str(row['Space Complexity Class']).replace('?', '')
            row['Time Complexity Class'] = str(row['Time Complexity Class']).replace('?', '')
            if '; ' in str(row['Variation']):
                dataframe = add_rows_for_algo(dataframe, row, index)
                found = True
                break
        if not found:
            searching = False
        else:
            found = False
    dataframe.to_csv('Analysis/data.csv')
    return dataframe

In [5]:
# pd.read_csv('Analysis/data_dirty.csv')

In [6]:
data = clean_data()

In [7]:
data

,Old Family #,Family Name,Looked at?,Variation,Algo ID,Algorithm Description,Final Call,Exact Problem Statement?,Exact algorithm?,Time Complexity (Average),...,n = 10^6 value,n = 10^6 scale,n = 10^6 Rate,n = 10^9 value,n = 10^9 scale,n = 10^9 Rate,Starting Complexity,Remarks,Papers for ratio evaluations,Domains
0,38.0,Optimal Binary Search Trees,2.0,Alphabetic Tree Problem,351.0,,,1,1,O(n),...,1000000,Inf,,1000000000,Inf,,1,,,Combinatorics
1,66.0,The Subset-Sum Problem,2.0,Subset Sum,546.0,,,1,1,O(nt/logt),...,,,,,,,1,,,Combinatorics
2,66.0,The Subset-Sum Problem,2.0,Subset Sum,547.0,,,1,1,O(n' t),...,,,,,,,1,Dynamic Programming,,Combinatorics
3,66.0,The Subset-Sum Problem,2.0,Subset Sum,548.0,,,1,1,O(n' t),...,,,,,,,1,,,Combinatorics
4,66.0,The Subset-Sum Problem,2.0,Subset Sum,549.0,,,1,1,O(σ^{(3/2)}),...,,,,,,,1,,,Combinatorics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,132.0,n-Queens Problem,2.0,Constructing solutions,965.0,,,1,1,O(n!),...,Inf,1,,Inf,1,,1,,,Combinatorics
706,132.0,n-Queens Problem,2.0,Counting Solutions,966.0,,,1,1,O(n!),...,Inf,1,,Inf,1,,1,,,Combinatorics
707,132.0,n-Queens Problem,2.0,Constructing solutions,966.0,,,1,1,O(n!),...,Inf,1,,Inf,1,,1,,,Combinatorics
708,132.0,n-Queens Problem,2.0,Counting Solutions,968.0,,,1,1,O(n!),...,,,,,,,1,,,Combinatorics


In [8]:
sorting = data[data["Variation"] == "Comparison Sorting"]

In [9]:
sorting = sorting.sort_values("Year")

In [10]:
sorting[["Year", "Time Complexity (Average)"]]

,Year,Time Complexity (Average)
11,1940,O( n² )
13,1945,O(n logn)
14,1956,O( n² )
123,1959,O( n² )
124,1960,O(n^1.5)
121,1961,O(n logn)
12,1962,O( n² )
16,1964,O(n logn)
125,1971,O(n log² n)
120,1986,O(n logn)


In [9]:
sorting.set_index("Year").to_dict()["Time Complexity (Average)"]

{1940: 'O( n² )',
 1945: 'O(n logn)',
 1956: 'O( n² )',
 1959: 'O( n² )',
 1960: 'O(n^1.5)',
 1961: 'O(n logn)',
 1962: 'O( n² )',
 1964: 'O(n logn)',
 1971: 'O(n log² n)',
 1986: 'O(n^1.33)',
 1997: 'O(n logn)',
 2002: 'O(n loglogn)'}

In [10]:
year_to_time =  {1940: lambda n: n**2, # 'O( n² )'
                 1945: lambda n: n*np.log(n), # 'O(n logn)'
                 1956: lambda n: n**2, # 'O( n² )'
                 1959: lambda n: n**2, # 'O( n² )'
                 1960: lambda n: n**1.5, # 'O(n^1.5)'
                 1961: lambda n: n*np.log(n), # 'O(n logn)'
                 1962: lambda n: n**2, # 'O( n² )'
                 1964: lambda n: n*np.log(n), # 'O(n logn)'
                 1971: lambda n: n*(np.log(n)**2), # 'O(n log² n)'
                 1986: lambda n: n**1.33, # 'O(n^1.33)'
                 1997: lambda n: n*np.log(n), # 'O(n logn)'
                 2002: lambda n: n*np.log(np.log(n))} # 'O(n loglogn)'

In [20]:
year_to_time_mono =  {1940: lambda n: n**2, # 'O( n² )'
                 1945: lambda n: n*np.log(n), # 'O(n logn)'
#                  1956: lambda n: n**2, # 'O( n² )'
#                  1959: lambda n: n**2, # 'O( n² )'
#                  1960: lambda n: n**1.5, # 'O(n^1.5)'
#                  1961: lambda n: n*np.log(n), # 'O(n logn)'
#                  1962: lambda n: n**2, # 'O( n² )'
#                  1964: lambda n: n*np.log(n), # 'O(n logn)'
#                  1971: lambda n: n*(np.log(n)**2), # 'O(n log² n)'
#                  1986: lambda n: n**1.33, # 'O(n^1.33)'
#                  1997: lambda n: n*np.log(n), # 'O(n logn)'
                 2002: lambda n: n*np.log(np.log(n))} # 'O(n loglogn)'

In [21]:
# Create figure
fig = go.Figure()

n = np.logspace(3, 18, base=10, axis=0, num=6)

# Add traces, one for each slider step
for step in n:
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            x=np.array([year for year in year_to_time_mono.keys()]),
            y=[func(step) for func in year_to_time_mono.values()], 
        )
    )

# Make 0th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "n = " + str(n[i]), "yaxis.title": "Runtime", "xaxis.title": "Year"},
              ],  # layout attribute
        label=str("n = 10^" + str(i + 3))
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders, yaxis_type="log")

fig.show()

In [15]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

n = np.logspace(3, 18, base=10, axis=0, num=6)

# Add traces, one for each slider step
for step in n:
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            x=np.array([year for year in year_to_time.keys()]),
            y=[func(step) for func in year_to_time.values()], 
        )
    )

# Make 0th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "n = " + str(n[i]), "yaxis.title": "Runtime", "xaxis.title": "Year"},
              ],  # layout attribute
        label=str("n = 10^" + str(i + 3))
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders, yaxis_type="log")

fig.show()

In [50]:
## generate HTML file
# import plotly.io as pio
# pio.write_html(fig, file='index.html', auto_open=True)